# Deep Nets using TF- Abstractions

[Prashant Brahmbhatt](www.github.com/hashbanger)

## Layers API

https://www.tensorflow.org/tutorials/layers

_____

### The Data

To compare these various abstractions we'll use a dataset easily available from the SciKit Learn library. The data is comprised of the results of a chemical analysis of wines grown in the same region in Italy by three different cultivators. There are thirteen different
measurements taken for different constituents found in the three types of wine. We will use the various TF Abstractions to classify the wine to one of the 3 possible labels.


In [0]:
from sklearn.datasets import load_wine

In [0]:
wine_data = load_wine()

In [0]:
type(wine_data)

sklearn.utils.Bunch

The sklearn Bunch is kind of a dictionary which contains the data as well as other information related to the data

In [0]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

Taking a look at the description of the data.

In [0]:
print(wine_data['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

____

Creating the features and the labels

In [0]:
feat_data = wine_data['data']

In [0]:
labels = wine_data['target']

### Splitting the dataset

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(feat_data, labels, test_size = 0.3, random_state = 101)

### Scaling the data

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
scaler = MinMaxScaler()

In [0]:
scaled_x_train = scaler.fit_transform(X_train)

In [0]:
scaled_x_test = scaler.fit_transform(X_test)

### One Hot Encoding the data

In [0]:
import pandas as pd

In [0]:
onehot_y_train = pd.get_dummies(y_train).values # or as_matrix()
# The layers API would require a numpy array rather than a dataframe
onehot_y_test = pd.get_dummies(y_test).values

_______

## The abstraction

### Parameters

In [0]:
num_feat = 13
num_hidden1 = 13
num_hidden2 = 13
num_outputs = 3
learning_rate = 0.01

In [0]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

### Placeholders

In [0]:
X = tf.placeholder(tf.float32, shape = [None, num_feat])
y_true = tf.placeholder(tf.float32, shape = [None, 3])

### Activation Function

In [0]:
actf = tf.nn.relu

### Create Layers

In [0]:
hidden1 = fully_connected(X, num_hidden1, activation_fn = actf)
hidden2 = fully_connected(hidden1, num_hidden2, activation_fn = actf)

In [0]:
output = fully_connected(hidden2, num_outputs)

### Loss Function

In [0]:
loss = tf.losses.softmax_cross_entropy(onehot_labels= y_true, logits = output)

### Optimizer

In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

### Init

In [0]:
init = tf.global_variables_initializer()

In [0]:
training_steps = 1000
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(training_steps):
        sess.run(train,feed_dict={X: scaled_x_train,y_true:onehot_y_train})
        
    # Get Predictions
    logits = output.eval(feed_dict={X:scaled_x_test})
    
    preds = tf.argmax(logits,axis=1)
    
    results = preds.eval()

Evaluating using the confusion matrix

In [0]:
from sklearn.metrics import confusion_matrix, classification_report

In [0]:
confusion_matrix(y_true= y_test, y_pred= results)

array([[19,  0,  0],
       [ 0, 22,  0],
       [ 0,  0, 13]])

In [0]:
print(classification_report(y_test, results))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        22
           2       1.00      1.00      1.00        13

   micro avg       1.00      1.00      1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54



So we got perfect accuracy

### de nada!